# Install Dependencies
Install all required Python packages for LLM agents, data processing, visualization, and network analysis.

In [ ]:
# Install required dependencies
%pip install -U langchain-community langchain_openai numpy pandas matplotlib seaborn nbformat ipykernel networkx imageio pillow scipy

# Import Libraries
Import core libraries: standard Python modules, data science tools (pandas, matplotlib), LangChain for LLM interaction, and NetworkX for multi-pool topology.

In [ ]:
# Standard library
import os
import sys
import time
import random
import re
import csv
import pickle
import statistics
import glob
import itertools
import logging
import io

# Third-party libraries
import requests
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import networkx as nx
from typing import List, Dict, Any, TextIO, Tuple, Set
from PIL import Image

# Project-specific packages
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage, BaseMessage

# Configure LLM Backend
Set up configuration for different LLM providers (OpenAI, Llama) and create the Llama API wrapper class with proper authentication.

In [ ]:
# Central Configuration

config: Dict[str, Any] = {
    "llm_choice": os.getenv("LLM_CHOICE", "llama"),
    "llama": {
        "model_name": os.getenv("LLAMA_MODEL_NAME", "meta-llama-3.3-70b-instruct-fp8"),
        "temperature": float(os.getenv("LLAMA_TEMP", 0.6)),
        "max_tokens": int(os.getenv("LLAMA_MAX_TOKENS", 500)),
        "api_url": os.getenv("LLAMA_API_URL"),
        "api_key": os.getenv("LLAMA_API_KEY")
    },
    "openai": {
        "model_name": os.getenv("OPENAI_MODEL_NAME", "gpt-4o-mini"),
        "temperature": float(os.getenv("OPENAI_TEMP", 1.0)),
        "max_tokens": int(os.getenv("OPENAI_MAX_TOKENS", 10)),
        "api_key": os.getenv("OPENAI_API_KEY")
    },
}

# Define Agent Classes
Create LLM-powered Agent and DummyAgent (always contributes 0) classes.

Agents make contribution decisions, maintain conversation history, and log all interactions.

In [ ]:
# -----------------------------
# Llama Helper Class
# -----------------------------
class Llama:
    def __init__(self, model_name: str, temperature: float, max_tokens: int, api_url: str, api_key: str = None) -> None:
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.api_url = api_url
        self.api_key = api_key or os.environ.get("LLAMA_API_KEY")

    def invoke(self, history: List[BaseMessage]) -> AIMessage:
        # Build messages, mapping LangChain roles to OpenAI roles
        messages: List[Dict[str, str]] = []
        for msg in history:
            if hasattr(msg, "content"):
                content = msg.content
            else:
                content = str(msg)

            # Normalize role
            if isinstance(msg, SystemMessage):
                role = "system"
            elif isinstance(msg, HumanMessage):
                role = "user"
            elif isinstance(msg, AIMessage):
                role = "assistant"
            else:
                raw = getattr(msg, "role", None)
                role = raw if raw in ("system", "user", "assistant") else "user"

            messages.append({"role": role, "content": content})

        data: Dict[str, Any] = {
            "model": self.model_name,
            "messages": messages,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature
        }

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        } if self.api_key else {}

        try:
            response = requests.post(self.api_url, json=data, headers=headers, timeout=30)
            response.raise_for_status()
        except requests.exceptions.Timeout:
            raise RuntimeError("Llama API request timed out.")
        except requests.exceptions.HTTPError as http_err:
            raise RuntimeError(f"Llama API HTTP error: {http_err}")
        except requests.exceptions.RequestException as err:
            raise RuntimeError(f"Llama API connection error: {err}")

        try:
            response_data = response.json()
            content = response_data["choices"][0]["message"]["content"]
        except (ValueError, KeyError, IndexError) as parse_err:
            raise RuntimeError(f"Failed to parse Llama API response: {parse_err}")

        return AIMessage(content=content)

def log_records(message: str, records_file: TextIO) -> None:
    """
    Helper function to log a message to records file so we can keep track of all history.
    """
    records_file.write(message + "\n")
    records_file.flush() # make sure it's written immediately

# Payoff calculation to handle multiple pools
def calculate_payoffs(contributions: List[float], e: float, m: float, na: int) -> List[float]:
    """
    Given a list of contributions from each agent, compute the payoff for each agent. For single pool case (backwards compatibility).
    """
    total: float = sum(contributions)
    shared_bonus: float = m * total / na
    return [e - c + shared_bonus for c in contributions]

def calculate_payoff_for_pool(contribs: List[float], m: float) -> float:
    """
    Calculate payoff bonus from a single pool for multi-pool games.
    """
    total = sum(contribs)
    return m * total / len(contribs) if contribs else 0.0

# -----------------------------
# Agent Class
# -----------------------------
class Agent:
    """
    Agent class for Public Goods Game participation. Uses configurable LLM backend (OpenAI or Llama). Maintains conversation history across rounds.
    Logs all interactions for experiment analysis. Responds to contribution prompts with <TOKEN>amount</TOKEN> format.
    """
    def __init__(self, name: str, system_message: str, records_file: TextIO) -> None:
        self.name = name
        # Start the conversation with a system message.
        self.history: List[BaseMessage] = [SystemMessage(content=system_message)]
        self.records_file = records_file

        # Log the creation of this agent and its system prompt to records.
        log_records(f"CREATING AGENT: {self.name}", records_file)
        log_records(f"System Prompt for {self.name}: {system_message}", records_file)

        # Create the appropriate LLM instance
        llm_choice = config["llm_choice"]
        if llm_choice == "llama":
            llama_cfg = config["llama"]
            self.llm = Llama(
                model_name=llama_cfg["model_name"],
                temperature=llama_cfg["temperature"],
                max_tokens=llama_cfg["max_tokens"],
                api_url=llama_cfg["api_url"],
                api_key=llama_cfg["api_key"]
            )
        else:
            openai_cfg = config["openai"]
            self.llm = ChatOpenAI(
                model_name=openai_cfg["model_name"],
                temperature=openai_cfg["temperature"],
                max_tokens=openai_cfg["max_tokens"],
                openai_api_key=openai_cfg["api_key"]
            )

    def chat(self, message: str) -> str:
        """
        Append a human message, call the LLM, append the assistant's reply,
        log everything, and return the response content.
        """
        log_records(f"{self.name} receives HUMAN message: {message}", self.records_file)
        self.history.append(HumanMessage(content=message))
        try:
            response = self.llm.invoke(self.history)
        except RuntimeError as err:
            log_records(f"{self.name} ERROR: {err}", self.records_file)
            return f"[ERROR]: {err}"

        # Store the response in the conversation history
        self.history.append(response)
        # Brief pause to help avoid rate limits
        log_records(f"{self.name} responds ASSISTANT: {response.content}", self.records_file)
        time.sleep(0.01)
        return response.content

# -----------------------------
# DummyAgent Class
# -----------------------------
class DummyAgent:
    """
    DummyAgent for robustness testing. Always contributes 0 tokens (simulates free-rider).
    """
    def __init__(self, name: str, system_message: str, records_file: TextIO) -> None:
        self.name = name
        self.history: List[BaseMessage] = [SystemMessage(content=system_message)]
        self.records_file = records_file

        log_records(f"CREATING DUMMY AGENT: {self.name}", records_file)
        log_records(f"(Dummy) System Prompt for {self.name}: {system_message}", records_file)

    def chat(self, message: str) -> str:
        """
        This agent always contributes "0".
        We still log the conversation but do not call any LLM.
        """
        log_records(f"{self.name} (dummy) receives HUMAN message: {message}", self.records_file)
        response_content = "<TOKEN>0</TOKEN>"
        log_records(f"{self.name} (dummy) responds ASSISTANT: {response_content}", self.records_file)
        time.sleep(0.01)
        return response_content

# Network and Pool Utilities
Functions to build pool structures for single-pool (global only) and multi-pool (global + smaller pools) game configurations.

Building pool topology enables studying cooperation across different network structures.

In [ ]:
def build_pools(num_agents: int, pool_sizes: List[int] = None) -> Dict[str, List[int]]:
    """
    Build pool structure. Single-pool: one global pool containing all agents. Multi-pool: global pool + local pools (subsets of agents).
    For 4 agents: 3 total pools, each agent belongs to exactly 2 pools (global + one local). Enables studying cooperation in complex network structures.
    """
    pools = {"global": list(range(num_agents))}

    if pool_sizes:  # Multi-pool case
        # Only support pool_sizes=[2] for simplicity
        if pool_sizes != [2]:
            raise ValueError("Only pool_sizes=[2] is supported in simplified version")

        # For 4 agents, create 2 non-overlapping pairs
        if num_agents != 4:
            raise ValueError("Multi-pool configuration currently only supports 4 agents")

        indices = list(range(num_agents))
        random.shuffle(indices)

        # Create 2 non-overlapping pairs (each agent in exactly one pair)
        for i in range(0, num_agents, 2):
            if i + 1 < num_agents:
                pair = [indices[i], indices[i + 1]]
                pool_id = f"group_2_{i//2}"
                pools[pool_id] = pair

    return pools

# Agent Utility Functions
Helper functions for experiment management: loading/saving checkpoints, generating system prompts, extracting and parsing contributions, and computing statistics.

In [ ]:
import os
from pathlib import Path

def get_experiment_directory(exp_type, story_name, na, nr, e, m, pool_sizes=None):
    """Generate standardized experiment directory path. Strucrure: experiments/{topology}/{exp_type}/ whre topology = "single_pool" or "multi_pool" """
    # Determine pool topology
    if pool_sizes:
        topology = "multi_pool"
    else:
        topology = "single_pool"

    # Build directory path: experiments/{topology}/{exp_type}/
    exp_dir = Path("experiments") / topology / exp_type

    # Create directory if it doesn't exist
    exp_dir.mkdir(parents=True, exist_ok=True)
    return exp_dir

def get_visualization_directory(exp_type, pool_sizes=None):
    """Generate visualization directory path"""
    # Determine pool topology
    if pool_sizes:
        topology = "multi_pool"
    else:
        topology = "single_pool"

    # Build directory path: experiments/vis/{topology}/{exp_type}/
    vis_dir = Path("experiments") / "vis" / topology / exp_type

    # Create directory if it doesn't exist
    vis_dir.mkdir(parents=True, exist_ok=True)
    return vis_dir

def get_visualization_filename(plot_type, exp_type, na, nr, e, m, pool_sizes=None, story_name=None):
    """Generate visualization filenames"""
    # Determine pool topology string
    if pool_sizes:
        if pool_sizes == [2]:
            pool_str = "multi_pool_2"
        else:
            raise ValueError("Only pool_sizes=[2] is supported in simplified version")
    else:
        pool_str = "single_pool"

    # Build parameter string
    params = f"ag{na}_ro{nr}_end{e}_mult{m}"

    # Build filename: {plot_type}_{pool_topology}_{exp_type}_{params}.pdf
    if exp_type == "different_story" or story_name is None:
        return f"{plot_type}_{pool_str}_{exp_type}_{params}.pdf"
    else:
        return f"{plot_type}_{pool_str}_{exp_type}_{story_name}_{params}.pdf"

def get_experiment_filename(file_type, exp_type, story_name, na, nr, e, m, pool_sizes=None, game_idx=None):
    """Generate standardized filenames"""
    # Determine pool topology string
    if pool_sizes:
        # SIMPLIFIED: only support pool_sizes=[2]
        if pool_sizes == [2]:
            pool_str = "multi_pool_2"
        else:
            raise ValueError("Only pool_sizes=[2] is supported in simplified version")
    else:
        pool_str = "single_pool"

    # Build parameter string
    params = f"ag{na}_ro{nr}_end{e}_mult{m}"

    # Build filename based on file type
    if file_type == "game_results":
        if exp_type == "different_story":
            return f"game_results_{pool_str}_{exp_type}_{params}.csv"
        else:
            return f"game_results_{pool_str}_{exp_type}_{story_name}_{params}.csv"

    elif file_type == "game_records":
        if exp_type == "different_story":
            base = f"game_records_{pool_str}_{exp_type}_{params}"
        else:
            base = f"game_records_{pool_str}_{exp_type}_{story_name}_{params}"

        if game_idx:
            return f"{base}_game{game_idx:03d}.txt"
        else:
            return f"{base}.txt"

    elif file_type == "checkpoint":
        if exp_type == "different_story":
            return f"checkpoint_{pool_str}_{exp_type}_{params}.pkl"
        else:
            return f"checkpoint_{pool_str}_{exp_type}_{story_name}_{params}.pkl"

    else:
        return f"{file_type}.txt"

def load_intermediate_results_new(exp_type, story_name, na, nr, e, m, pool_sizes=None):
    """Load intermediate results for resuming interrupted experiments."""
    exp_dir = get_experiment_directory(exp_type, story_name, na, nr, e, m, pool_sizes)
    checkpoint_filename = get_experiment_filename("checkpoint", exp_type, story_name, na, nr, e, m, pool_sizes)
    new_path = exp_dir / checkpoint_filename

    # Check new location first
    if new_path.exists():
        with open(new_path, 'rb') as f:
            results = pickle.load(f)
        print(f"Loaded checkpoint from new location: {new_path}")
        return results, new_path

    # Backward compatibility: check old location pattern (from original notebook)
    old_filename = f"results_{exp_type}_{story_name}_ag{na}_ro{nr}_end{e}_mult{m}"
    if pool_sizes:
        old_filename += f"_pools{'-'.join(map(str, pool_sizes))}"
    old_filename += ".pkl"

    if os.path.exists(old_filename):
        with open(old_filename, 'rb') as f:
            results = pickle.load(f)
        print(f"Loaded checkpoint from old location: {old_filename}")
        # Migrate to new location
        with open(new_path, 'wb') as f:
            pickle.dump(results, f)
        print(f"Migrated checkpoint to: {new_path}")
        return results, new_path

    # No existing results
    print(f"No existing checkpoint found. Starting fresh at: {new_path}")
    return {} if exp_type != "different_story" else [], new_path

def save_intermediate_results_new(results, checkpoint_path):
    """Save intermediate results to checkpoint file for resumability."""
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(results, f)

# These functions remain the same - no changes needed:
def get_system_prompt(na, e, m, is_networked=False):
    """
    Generate system prompt for agents based on game configuration.
    Args:
        na: Number of agents
        e: Token endowment per round
        m: Multiplier for shared pool returns
        is_networked: True for multi-pool, False for single-pool
    Returns:
        System prompt template with STORY placeholder
    """
    if is_networked:
        base = (
            f"You are participating in a repeated networked public goods game with {na} agents. "
            f"In each round, you receive {e} tokens. "
            "You will be prompted sequentially for contributions to each shared pool - some with all agents (global) and some with smaller subsets. "
            f"After everyone contributes, each pool's total is multiplied by {m} and divided equally among its members. "
            "Respond only with <TOKEN>k</TOKEN> where k is your integer contribution to the current pool."
        )
    else:
        base = (
            f"You are participating in a repeated public goods game with {na - 1} other agents. "
            f"In each round, you receive {e} tokens. You must decide how many tokens (an integer between 0 and {e}) "
            f"to contribute to a public pool. After everyone contributes, the total contribution is multiplied by {m} "
            f"and divided equally among all {na} agents. This means your payoff for a round is calculated as: \\n\\n"
            f"    {e} - (your contribution) + ({m} * total contributions / {na})\\n\\n"
            "Please respond **only** with the number of tokens you wish to contribute for this round, enclosed within `<TOKEN>` and `</TOKEN>` tags. "
            "For example: `<TOKEN>5</TOKEN>`. Do not provide any additional text, explanations, or summaries. "
        )
    return base + "STORY"

def compute_and_print_statistics(scores_by_prompt, exp_name):
    """Computes and prints mean and standard deviation of scores for each prompt."""
    print(f"\\n=== Collaboration Scores for {exp_name} ===")
    for prompt_label, score_list in scores_by_prompt.items():
        mean_val = statistics.mean(score_list)
        stdev_val = statistics.stdev(score_list) if len(score_list) > 1 else 0
        print(f"{prompt_label}: Mean = {mean_val:.4f}, SD = {stdev_val:.4f}")

def extract_contribution(response_str):
    """Extract valid contribution from agent's response."""
    match = re.search(r"<TOKEN>(\d+)</TOKEN>", response_str)
    if match:
        return int(match.group(1))
    return None

def get_valid_contribution(agent, round_num, e, max_retries=5, pool_context=""):
    """Get valid contribution from agent with retries."""
    retries = 0
    while retries < max_retries:
        prompt = f"Round {round_num}: {pool_context}What is your contribution (0-{e})?"

        if retries > 0:
            prompt += " Your previous response was invalid. **Only provide a number inside `<TOKEN>...</TOKEN>`** with no extra text. Example: `<TOKEN>5</TOKEN>`."

        response_str = agent.chat(prompt).strip()
        print(f"{agent.name} response (attempt {retries + 1}): {response_str}")

        contribution = extract_contribution(response_str)

        if contribution is not None:
            return contribution

        print(f"Warning: {agent.name} provided an invalid response. Retrying... ({retries + 1}/{max_retries})")
        retries += 1

    print(f"Error: {agent.name} failed to provide a valid response after {max_retries} attempts. Defaulting to 0.")
    return 0

# Game Logic Functions
Core game mechanics: collects contributions, calculates payoffs, and logs data in consistent CSV format for downstream analysis.

In [ ]:
CSV_HEADER = [
    "Game", "PromptType", "Round", "AgentName",
    "Contribution",      # Legacy column: total contribution amount
    "GlobalContrib",     # Global pool contribution
    "LocalContrib",      # Sum of local pool contributions
    "TotalContrib",      # Total across all pools (Global + Local)
    "RoundPayoff", "CumulativePayoff", "CollaborationScore"
]

def collect_contributions_single_pool(agents, round_num, e):
    """Collects valid contributions from all agents for single pool game."""
    contributions = []
    for agent in agents:
        contribution = get_valid_contribution(agent, round_num, e)

        # Enforce valid contribution range
        if contribution > e:
            print(f"{agent.name} attempted to contribute {contribution} tokens but only has {e}. "
                    f"Reducing contribution to {e}.")
            contribution = e
        contribution = max(0, contribution)
        contributions.append(contribution)
    print(f"Round Contributions: {contributions}")
    return contributions

def collect_contributions_multi_pool(agents: List[Agent], pools: Dict[str, List[int]], e: int) -> Dict[str, Dict[int, int]]:
    """Collects contributions from agents for multi-pool games."""
    remaining = {i: e for i in range(len(agents))}
    contributions = {pid: {} for pid in pools}

    # Randomize pool order (agents don't know order)
    for pid in random.sample(list(pools.keys()), len(pools)):
        # Randomize agent order within pool
        members = random.sample(pools[pid], len(pools[pid]))
        for i in members:
            others = [agents[j].name for j in pools[pid] if j != i]
            pool_context = f"Pool '{pid}' with {others}. You have {remaining[i]} tokens remaining. "

            contribution = get_valid_contribution(
                agent=agents[i],
                round_num=0,  # Will be set by caller
                e=remaining[i],
                pool_context=pool_context
            )
            contribution = max(0, min(contribution, remaining[i]))
            remaining[i] -= contribution
            contributions[pid][i] = contribution

    return contributions

def calculate_rewards_single_pool(contributions, agents, e, m, na, total_rewards, round_num):
    """Calculates payoffs and updates agent rewards for single pool."""
    round_total = sum(contributions)
    # Calculate payoffs for the round using original formula
    payoffs = calculate_payoffs(contributions, e, m, na)
    print(f"Round Payoffs: {payoffs}")

    for idx, agent in enumerate(agents):
        total_rewards[idx] += payoffs[idx]
        summary = (
            f"Round {round_num} Summary:\\n"
            f" - Your contribution: {contributions[idx]}\\n"
            f" - Total contributions: {round_total}\\n"
            f" - Your payoff this round: {payoffs[idx]:.2f}\\n"
            f" - Your cumulative reward: {total_rewards[idx]:.2f}"
        )
        agent.chat(summary)

    return payoffs, round_total, total_rewards

def calculate_rewards_multi_pool(
    contributions: Dict[str, Dict[int,int]],
    pools: Dict[str, List[int]],
    agents: List[Agent],
    e: int,
    m: float,
    total_rewards: List[float],
    round_num: int
) -> List[float]:
    """
    Calculates payoffs using the formula: π_i = Σ_{p: i ∈ M_p} (m * T_p / |M_p|) + (T - Σ_p t_{i,p})
    """
    n = len(agents)
    round_payoffs = [0.0] * n

    # First term: Σ_{p: i ∈ M_p} (m * T_p / |M_p|)
    for pid, pool_contributions in contributions.items():
        members = list(pool_contributions.keys())
        pool_total = sum(pool_contributions.values())
        pool_size = len(members)

        if pool_size > 0:
            bonus_per_member = m * pool_total / pool_size
            for agent_idx in members:
                round_payoffs[agent_idx] += bonus_per_member

    # Second term: (T - Σ_p t_{i,p}) where T = e (endowment)
    for i in range(n):
        total_contributed = sum(contributions[pid].get(i, 0) for pid in contributions)
        kept_tokens = e - total_contributed
        round_payoffs[i] += kept_tokens
        total_rewards[i] += round_payoffs[i]

    # Send feedback to agents
    for i, agent in enumerate(agents):
        msg = [f"Round {round_num} Results:"]
        agent_pools = []
        for pid, pool_contribs in contributions.items():
            if i in pool_contribs:
                others_in_pool = [j for j in pools[pid] if j != i]
                other_contributions = [f"{agents[j].name}→{pool_contribs[j]}" for j in others_in_pool if j in pool_contribs]
                msg.append(f"Pool '{pid}': You→{pool_contribs[i]}, " + ", ".join(other_contributions))

        msg.append(f"Round payoff: {round_payoffs[i]:.2f}, Cumulative: {total_rewards[i]:.2f}")
        agent.chat("\\n".join(msg))

    return round_payoffs

def get_csv_header():
    """Returns the unified CSV header for all experiment types."""
    return CSV_HEADER

def log_round_results(
    csv_writer,
    agents,
    game_index,
    prompt_label,
    round_num,
    payoffs,
    total_rewards,
    exp_type,
    # Single-pool specific
    contributions_single=None,
    # Multi-pool specific
    pool_contributions=None
):
    """
    Log round results to CSV in unified format supporting both pool topologies.

    Single-pool: Uses contributions_single parameter
    Multi-pool: Uses pool_contributions parameter (dict by pool_id)

    Maintains consistent column structure for easy analysis across experiment types.
    """
    for idx, agent in enumerate(agents):
        # Determine story label based on experiment type
        if exp_type == "different_story" and hasattr(agent, 'story_label'):
            story_label = agent.story_label
        else:
            story_label = prompt_label

        if pool_contributions:  # Multi-pool case
            global_contrib = pool_contributions.get("global", {}).get(idx, 0)
            local_contrib = sum(
                pool_contributions[pid].get(idx, 0)
                for pid in pool_contributions
                if pid != "global"
            )
            total_contrib = global_contrib + local_contrib
            # For backward compatibility
            contribution = total_contrib

        else:  # Single-pool case
            contribution = contributions_single[idx] if contributions_single else 0
            global_contrib = contribution  # In single-pool, all goes to "global"
            local_contrib = 0
            total_contrib = contribution

        csv_writer.writerow([
            game_index,
            story_label,
            round_num,
            agent.name,
            contribution,      # Backward compatibility column
            global_contrib,
            local_contrib,
            total_contrib,
            f"{payoffs[idx]:.2f}",
            f"{total_rewards[idx]:.2f}",
            ""  # CollaborationScore (empty for per-round rows)
        ])

def log_final_score(csv_writer, game_index, prompt_label, effective_score):
    """Log the final collaboration score row."""
    csv_writer.writerow([
        game_index,
        prompt_label,
        "final",
        "All",
        "",  # Contribution
        "",  # GlobalContrib
        "",  # LocalContrib
        "",  # TotalContrib
        "",  # RoundPayoff
        "",  # CumulativePayoff
        f"{effective_score:.4f}"  # CollaborationScore
    ])

# Main Game Execution
Functions to run complete games with multiple rounds, handle both single-pool and multi-pool experiment types, and manage the overall game flow and data collection.

In [ ]:
"""
Execute complete game sessions with multiple rounds.

Features:
- Handles both single-pool and multi-pool topologies seamlessly
- Collects agent contributions and calculates payoffs
- Provides round-by-round feedback to agents
- Logs detailed results with descriptive filenames
- Saves data in consistent CSV format for analysis
"""

def execute_game_rounds(agents, na, nr, e, m, exp_dir, game_index, prompt_label, exp_type, num_dummy_agents, pools=None, csv_filename=None):
    is_networked = pools is not None and len(pools) > 1
    total_rewards = [0 for _ in range(na)]
    total_game_contributions = 0

    # Open CSV file in experiment directory with descriptive filename
    csv_path = exp_dir / csv_filename
    csv_exists = csv_path.exists()

    with open(csv_path, "a", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)

        # Write header if new file
        if not csv_exists:
            writer.writerow(CSV_HEADER)

        print("\\n=== Starting a New Game ===")
        for round_num in range(1, nr + 1):
            print(f"\\n--- Round {round_num} ---")

            if is_networked:
                # Multi-pool game
                pool_contributions = collect_contributions_multi_pool(agents, pools, e)
                payoffs = calculate_rewards_multi_pool(pool_contributions, pools, agents, e, m, total_rewards, round_num)
                log_round_results(
                    writer, agents, game_index, prompt_label, round_num,
                    payoffs, total_rewards, exp_type, pool_contributions=pool_contributions
                )
                total_game_contributions += sum(sum(v.values()) for v in pool_contributions.values())
            else:
                # Single-pool game
                contributions = collect_contributions_single_pool(agents, round_num, e)
                payoffs, round_total, total_rewards = calculate_rewards_single_pool(contributions, agents, e, m, na, total_rewards, round_num)
                log_round_results(
                    writer, agents, game_index, prompt_label, round_num,
                    payoffs, total_rewards, exp_type, contributions_single=contributions
                )
                total_game_contributions += round_total

        # Calculate and log final score
        if is_networked:
            max_possible = na * e * nr  # Total possible contributions
        else:
            max_possible = (na - num_dummy_agents) * e * nr

        effective_score = total_game_contributions / max_possible
        print(f"\\nEffective Collaboration Score: {effective_score:.4f}")

        # Log final score
        log_final_score(writer, game_index, prompt_label, effective_score)

    return effective_score, total_rewards

def run_single_game(game_index: int, prompt_label: str, system_prompt_used: str,
                    na: int, nr: int, e: int, m: float, exp_dir, exp_type: str, story_name: str,
                    num_dummy_agents, pools=None, pool_sizes=None) -> float:
    """
    Run a single game.

    Creates fresh agent instances, runs all rounds, calculates final collaboration score.
    Automatically handles single-pool vs multi-pool logic based on pools parameter.

    Returns:
        Final collaboration score for the game
    """
    # Generate descriptive records filename
    records_filename = get_experiment_filename("game_records", exp_type, story_name, na, nr, e, m, pool_sizes, game_index)
    records_path = exp_dir / records_filename

    # Generate descriptive CSV filename
    csv_filename = get_experiment_filename("game_results", exp_type, story_name, na, nr, e, m, pool_sizes)

    with open(records_path, "w", encoding="utf-8") as records_file:
        # Create new agents for this game
        agents = []
        for i in range(na):
            if i < num_dummy_agents:
                agent = DummyAgent(f"Agent_{i+1}", system_prompt_used, records_file)
            else:
                agent = Agent(f"Agent_{i+1}", system_prompt_used, records_file)
            agents.append(agent)

        for agent in agents:
            agent.story_label = prompt_label

        # Execute all rounds of the game
        effective_score, _ = execute_game_rounds(
            agents, na, nr, e, m, exp_dir, game_index, prompt_label, exp_type, num_dummy_agents, pools, csv_filename
        )

    return effective_score

def run_single_game_random_story(game_index: int, system_prompt_story: str, na: int, nr: int, e: int, m: float,
                                exp_dir, exp_type: str, story_prompts: dict, pools=None, pool_sizes=None) -> (float, list):
    """
    Run a single game where each agent gets a random story.
    """
    # Generate descriptive records filename
    records_filename = get_experiment_filename("game_records", exp_type, None, na, nr, e, m, pool_sizes, game_index)
    records_path = exp_dir / records_filename

    # Generate descriptive CSV filename
    csv_filename = get_experiment_filename("game_results", exp_type, None, na, nr, e, m, pool_sizes)

    with open(records_path, "w", encoding="utf-8") as records_file:
        agents = []

        # Create agents with random story prompts
        for i in range(na):
            chosen_label, chosen_story = random.choice(list(story_prompts.items()))
            prompt_text = system_prompt_story.replace("STORY", chosen_story)
            agent = Agent(f"Agent_{i+1}", prompt_text, records_file)
            agent.story_label = chosen_label
            agents.append(agent)

        # Execute all rounds of the game
        effective_score, total_rewards = execute_game_rounds(
            agents, na, nr, e, m, exp_dir, game_index, "All", exp_type, 0, pools, csv_filename
        )

        # Prepare results: (agent_name, story_label, cumulative_reward)
        agent_results = [(agents[i].name, agents[i].story_label, total_rewards[i]) for i in range(na)]

    return effective_score, agent_results

# Experiment Configuration
High-level experiment runners for three experiment types: Homogeneous (same story), Robustness (with dummy agent), Heterogeneous (random stories).

Supports both pool topologies.

In [ ]:
def run_same_story_experiment(is_bad_apple, story_index, num_rounds_list, endowment_list, multiplier_list, num_games, num_agents_list, exp_type, pool_sizes=None):
    """Runs the same story experiment where all agents receive the same story."""
    story_files = sorted(glob.glob("stories/*.txt"))
    if story_index >= len(story_files):
        print("Invalid story index. Exiting.")
        sys.exit(1)

    selected_story_file = story_files[story_index]
    story_name = os.path.splitext(os.path.basename(selected_story_file))[0]

    for na, nr, e, m in itertools.product(num_agents_list, num_rounds_list, endowment_list, multiplier_list):
        # Determine experiment type name
        base_exp_type = "bad_apple" if is_bad_apple else "same_story"
        num_dummy_agents = 1 if is_bad_apple else 0

        print(f"\\n\\n######################")
        print(f"Running {base_exp_type} experiment: {story_name}")
        print(f"Agents: {na}, Rounds: {nr}, Endowment: {e}, Multiplier: {m}")
        if pool_sizes:
            print(f"Pool sizes: {pool_sizes}")
        print(f"######################\\n")

        scores_by_prompt = run_same_story_games(
            base_exp_type, selected_story_file, story_name, na, nr, e, m, num_games, num_dummy_agents,
            base_exp_type, pool_sizes  # Pass base_exp_type as game_exp_type parameter
        )
        compute_and_print_statistics(scores_by_prompt, f"{base_exp_type}_{story_name}")

def run_different_story_experiment(num_rounds_list, endowment_list, multiplier_list, num_games, num_agents_list, exp_type, pool_sizes=None):
    """Runs different story experiment where each agent receives a random story."""

    for na, nr, e, m in itertools.product(num_agents_list, num_rounds_list, endowment_list, multiplier_list):
        print(f"\\n\\n######################")
        print(f"Running different_story experiment")
        print(f"Agents: {na}, Rounds: {nr}, Endowment: {e}, Multiplier: {m}")
        if pool_sizes:
            print(f"Pool sizes: {pool_sizes}")
        print(f"######################\\n")

        story_prompts = load_all_story_prompts()
        system_prompt = get_system_prompt(na, e, m, pool_sizes is not None)

        scores_list, rewards_by_story = run_different_story_games(
            na, nr, e, m, num_games, story_prompts, system_prompt, exp_type, pool_sizes
        )

        print(f"\\n=== Rewards by Story for different_story experiment ===")
        compute_and_print_statistics(rewards_by_story, "different_story")
        if scores_list:
            print(f"\\nOverall Effective Collaboration Score: Mean = {statistics.mean(scores_list):.4f}, SD = {statistics.stdev(scores_list):.4f}")

def run_same_story_games(exp_type, story_file, story_name, na, nr, e, m, num_games, num_dummy_agents, game_exp_type, pool_sizes=None):
    """
    Execute multiple games with identical story prompts for all agents.
    Supports checkpoint/resume functionality for long experiments.
    Automatically generates descriptive filenames for data organization.
    """

    # Get experiment directory
    exp_dir = get_experiment_directory(exp_type, story_name, na, nr, e, m, pool_sizes)

    # Load story content
    with open(story_file, "r", encoding="utf-8") as f:
        story_content = f.read()
    if story_name not in ["maxreward", "noinstruct"]:
        story_content = "Your behavior is influenced by the following bedtime story your mother read to you every night: " + story_content

    # Create system prompt
    is_networked = pool_sizes is not None
    prompt_text = get_system_prompt(na, e, m, is_networked).replace("STORY", story_content)

    # Load existing results with backward compatibility
    intermediate_results, checkpoint_path = load_intermediate_results_new(exp_type, story_name, na, nr, e, m, pool_sizes)

    if story_name not in intermediate_results:
        intermediate_results[story_name] = []

    scores = intermediate_results[story_name][:]
    print(f"\\n=== Running Games: {story_name} ===")
    print(f"Experiment directory: {exp_dir}")
    print(f"Completed games: {len(scores)}")

    for game_index in range(len(scores) + 1, num_games + 1):
        print(f"\\n=== Game {game_index}/{num_games} ({story_name}) ===")

        # Build pools if networked
        pools = build_pools(na, pool_sizes) if is_networked else None

        try:
            score = run_single_game(
                game_index, story_name, prompt_text, na, nr, e, m, exp_dir,
                exp_type, story_name, num_dummy_agents, pools, pool_sizes
            )
            scores.append(score)
            intermediate_results[story_name].append(score)
            save_intermediate_results_new(intermediate_results, checkpoint_path)
            print(f"Game {game_index} completed. Score: {score:.4f}")

        except Exception as exc:
            print(f"Game {game_index} failed: {exc}")
            continue

    return {story_name: scores}

def run_different_story_games(na, nr, e, m, num_games, story_prompts, system_prompt, exp_type, pool_sizes=None):
    """Run different story games."""

    # Get experiment directory
    exp_dir = get_experiment_directory("different_story", None, na, nr, e, m, pool_sizes)

    # Load existing results
    intermediate_results, checkpoint_path = load_intermediate_results_new("different_story", None, na, nr, e, m, pool_sizes)

    scores_list = []
    rewards_by_story = {story: [] for story in story_prompts}

    print(f"\\n=== Running Different Story Games ===")
    print(f"Experiment directory: {exp_dir}")
    print(f"Completed games: {len(intermediate_results)}")

    for game_index in range(len(intermediate_results) + 1, num_games + 1):
        print(f"\\n=== Game {game_index}/{num_games} (different_story) ===")

        # Build pools if networked
        pools = build_pools(na, pool_sizes) if pool_sizes else None

        try:
            score, agent_results = run_single_game_random_story(
                game_index, system_prompt, na, nr, e, m, exp_dir, exp_type, story_prompts, pools, pool_sizes
            )

            # Store results
            intermediate_results.append((game_index, score, agent_results))
            save_intermediate_results_new(intermediate_results, checkpoint_path)

            scores_list.append(score)
            for _, story_label, reward in agent_results:
                rewards_by_story[story_label].append(reward)

            print(f"Game {game_index} completed. Score: {score:.4f}")

        except Exception as exc:
            print(f"Game {game_index} failed: {exc}")
            continue

    return scores_list, rewards_by_story

def load_all_story_prompts(stories_dir="stories"):
    """Load all story prompts from files."""
    story_prompts = {}
    for story_file in sorted(glob.glob(f"{stories_dir}/*.txt")):
        story_name = os.path.splitext(os.path.basename(story_file))[0]
        with open(story_file, "r", encoding="utf-8") as f:
            content = f.read()
        if story_name not in ["maxreward", "noinstruct"]:
            content = "Your behavior is influenced by the following bedtime story your mother read to you every night: " + content
        story_prompts[story_name] = content
    return story_prompts

# Main Execution Block
Main experiment runner function: run any experiment type with optional pool_sizes parameter.

Use pool_sizes=None for single-pool or pool_sizes=[2] for multi-pool mode.

In [ ]:
# -----------------------------
# Configurable Run Function
# -----------------------------

def run_experiment(exp_type, story_index=0, pool_sizes=None):
    """
    Main experiment entry point supporting all experiment types and pool topologies.

    Args:
        exp_type: "same_story", "bad_apple", "different_story"
        story_index: Index of story to use (for same_story/bad_apple)
        pool_sizes: None for single-pool, [2] for multi-pool pairs
    """
    # Experiment configurations
    num_rounds_list = [5]
    endowment_list = [10]
    multiplier_list = [1.5]

    try:
        if exp_type in ["same_story", "bad_apple"]:
            num_games = 100
            num_agents_list = [4, 16, 32] if exp_type == "same_story" else [4]
            run_same_story_experiment(
                is_bad_apple=(exp_type == "bad_apple"),
                story_index=story_index,
                num_rounds_list=num_rounds_list,
                endowment_list=endowment_list,
                multiplier_list=multiplier_list,
                num_games=num_games,
                num_agents_list=num_agents_list,
                exp_type=exp_type,
                pool_sizes=pool_sizes
            )
        elif exp_type == "different_story":
            num_games = 400
            num_agents_list = [4]
            run_different_story_experiment(
                num_rounds_list, endowment_list, multiplier_list, num_games, num_agents_list, exp_type, pool_sizes
            )
        else:
            print(f"[ERROR] Unknown experiment type: {exp_type}")
    except Exception as e:
        print(f"[ERROR] Experiment '{exp_type}' failed: {e}")

# Single-Pool Experiments – Homogeneous Agents
Baseline experiments: All agents get the same story prompt. Tests how different narratives affect cooperation in standard public goods setting.

Run all 12 stories with same-story configuration across different agent sizes (4, 16, 32 agents) for 100 games each.

In [ ]:
# Execute homogeneous experiments: all agents receive identical story prompts
# Tests baseline cooperation patterns for each of the 12 story types

for i in range(12):
    print(f"Running same_story experiment for story {i}")
    run_experiment("same_story", story_index=i)

# Single-Pool Experiments – Robustness Testing

Same as homogeneous but with one dummy agent (always contributes 0) to test cooperation resilience (robustness) to non-cooperative agents.

In [ ]:
# Execute robustness experiments: same as homogeneous but includes one non-cooperative agent
# Tests resilience of cooperation when facing persistent free-riding behavior

for i in range(12):
    print(f"Running bad_apple experiment for story {i}")
    run_experiment("bad_apple", story_index=i)

# Single-Pool Experiments – Heterogeneous Agents

Run experiments where each agent gets a random story from the story corpus, testing performance in mixed narrative environments.

In [ ]:
# Execute heterogeneous experiment: each agent receives random story prompt
# Tests cooperation patterns when agents have diverse narrative influences

print("Running different_story experiment")
run_experiment("different_story")

# Multi-Pool Experiments

Test cooperation in complex networks. Agents contribute to both global pool (all agents) and local pools (subsets).

Reveals how network structure affects behavior.

In [ ]:
# Multi-pool topology: global pool (all agents) + local pools (agent subsets)

pool_sizes = [2]  # Creates 2 non-overlapping pools of size 2, plus 1 global pool (3 pools total)

# Homogeneous multi-pool
for i in range(12):
    print(f"Running networked same_story experiment for story {i}")
    run_experiment("same_story", story_index=i, pool_sizes=pool_sizes)


In [ ]:
# Multi-pool topology: global pool (all agents) + local pools (agent subsets)

pool_sizes = [2]  # Creates 2 non-overlapping pools of size 2, plus 1 global pool (3 pools total)

# Heterogeneous multi-pool
print("Running networked different_story experiment")
run_experiment("different_story", pool_sizes=pool_sizes)

# Single-Pool Experiments – Visualization
Data loading, preprocessing, and visualization functions for single-pool experiments: violin plots, scaling analysis, and statistical comparisons.

## Data Loading and Preprocessing
Load CSV files, filter data by experiment type, and prepare datasets for visualization.

In [ ]:
def load_csv_files(pattern):
    """Loads all CSV files matching a pattern and merges them into a DataFrame."""
    files = glob.glob(pattern)
    if not files:
        print(f"No files found for pattern: {pattern}")
        return pd.DataFrame()
    return pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

def preprocess_data(df, metric):
    """
    Prepares data by filtering only final round rows and converting columns to numeric types.
    Metric can be 'CollaborationScore' or 'CumulativePayoff'.
    """
    if df is None or df.empty:
        return None

    if metric == "CollaborationScore":
        df = df[df["Round"] == "final"].copy()
    else:  # "CumulativePayoff"
        df = df[df["Round"] != "final"].copy()
        df["Round"] = pd.to_numeric(df["Round"], errors="coerce")
        df.dropna(subset=["Round"], inplace=True)
        df = df.loc[df.groupby(["Game", "AgentName"])["Round"].idxmax()].copy()

    df[metric] = pd.to_numeric(df[metric], errors="coerce")
    df.dropna(subset=[metric], inplace=True)

    return df

## File-Pattern Configurations
Define file patterns and color schemes for consistent visualization across all experiment types.

In [ ]:
AGENT_SIZES = [4, 16, 32]

# Define baseline and meaningful story prompts
BASELINE_STORIES = ["noinstruct", "nsCarrot", "maxreward", "nsPlumber"]
MEANINGFUL_STORIES = ["OldManSons", "Odyssey", "Soup", "Peacemaker","Musketeers", "Teamwork", "Spoons", "Turnip"]

# Define color gradients for baseline (blue) and meaningful stories (pink)
BLUE_SHADES = ["#87CEFA", "#4682B4", "#4169E1", "#27408B"]  # Light to Dark Blue
PINK_SHADES = ["#FFB3E6", "#FF99CC", "#FF66B3", "#FF4D9E", "#F02278", "#D81B60", "#B83B7D", "#B22272"]  # Light to Dark Pink

# Define color dictionary for plot consistency
COLOR_DICT = {
    # Baseline condition (Shades of Blue)
    "maxreward": "#87CEFA",
    "noinstruct": "#4682B4",
    "nsCarrot": "#4169E1",
    "nsPlumber": "#27408B",
    # Meaningful stories (Shades of Purple/Pink)
    "Odyssey": "#FFB3E6",
    "Soup": "#FF99CC",
    "Peacemaker": "#FF66B3",
    "Musketeers": "#FF4D9E",
    "Teamwork": "#F02278",
    "Spoons": "#D81B60",
    "Turnip": "#B83B7D",
    "OldManSons": "#B22272",
}

VISUALISATION_EXPERIMENTS = {
    "same_story": dict(
        sizes=AGENT_SIZES,
        pattern="experiments/single_pool/same_story/game_results_single_pool_same_story_*_ag{N}_ro5_end10_mult1.5.csv"
    ),
    "different_story": dict(
        sizes=[4],
        pattern="experiments/single_pool/different_story/game_results_single_pool_different_story_ag{N}_ro5_end10_mult1.5.csv"
    ),
    "bad_apple": dict(
        sizes=[4],
        pattern="experiments/single_pool/bad_apple/game_results_single_pool_bad_apple_*_ag{N}_ro5_end10_mult1.5.csv"
    ),
}

CATEGORIES_FOR_VIOLIN = {
    f"{name}_{N}_agents": exp["pattern"].format(N=N)
    for name, exp in VISUALISATION_EXPERIMENTS.items()
    for N in exp["sizes"]
}

CATEGORY_GROUPS = {
    "temp_0.6": {
        N: VISUALISATION_EXPERIMENTS["same_story"]["pattern"].format(N=N)
        for N in AGENT_SIZES
    }
}

# For Latex Table
TABLE_KEYS = {
    "same_story": "homogeneous",
    "different_story": "heterogeneous",
    "bad_apple": "robustness",
}

# For table generation
CATEGORIES_FOR_TABLE = {
    TABLE_KEYS[name]: [
        VISUALISATION_EXPERIMENTS[name]["pattern"].format(N=N)
        for N in VISUALISATION_EXPERIMENTS[name]["sizes"]
    ]
    for name in VISUALISATION_EXPERIMENTS
}

In [ ]:
CATEGORIES_FOR_VIOLIN

In [ ]:
CATEGORY_GROUPS

In [ ]:
CATEGORIES_FOR_TABLE

## 1. Distribution Analysis Plots
Generate violin plots for different experiment types:

Collaboration Score for Homogenous and Robustness experiments. Payoff per Agent for Heterogenous experiment.

In [ ]:
# Ensure vectorized rendering
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

def plot_violin(df, metric, title, output_pdf, plot_mean_line=True, show_legend=False):
    """
    Generate violin plots showing distribution of collaboration scores or payoffs.
    Features:
    - Orders stories by mean performance (low to high)
    - Color-codes baseline vs meaningful stories
    - Overlays mean trend line
    - Saves as PDF
    """
    if df is None or df.empty:
        print(f"No data to visualize for {title}")
        return

    # Compute x-axis order based on mean metric values
    order = (df.groupby("PromptType")[metric]
            .mean()
            .sort_values(ascending=True)
            .index.tolist())

    plt.figure(figsize=(12, 7))

    # Define color palette
    palette = {cat: COLOR_DICT.get(cat, "#888888") for cat in order}

    # Violin plot with embedded box plot
    ax = sns.violinplot(
        data=df,
        x="PromptType",
        y=metric,
        hue="PromptType",
        palette=palette,
        inner="box",
        dodge=False,
        order=order,
        bw_adjust=5, # Adjusting KDE bandwidth
        scale="area", # Uniform width across all violins
    )

    if ax.get_legend():
        ax.get_legend().remove()

    # Overlay scatter points
    sns.stripplot(
        data=df,
        x="PromptType",
        y=metric,
        color="black",
        dodge=False,
        alpha=0.2,
        size=2,
        zorder=2,
        order=order
    )

    # (Optional) Plot mean trend line
    if plot_mean_line:
        means = df.groupby("PromptType")[metric].mean().loc[order]
        x_positions = list(range(len(order)))
        plt.plot(
            x_positions, means.values, marker='o',
            color='black', linestyle='-', linewidth=2,
            markersize=6, alpha=0.5, label="Mean Trend"
        )
        if show_legend:
            plt.legend(["Mean Trend"])

    # Customize plot
    if metric == "CollaborationScore":
        plt.ylim(0, 1.3)  # Set range for Collaboration Score
    elif metric == "CumulativePayoff":
        plt.ylim(0, 120) #Set range for Cumulative Payoff
    plt.xlabel("Story Prompt", fontsize=18, labelpad=15)

    ylabel_text = "Payoff per Agent" if metric == "CumulativePayoff" else "Collaboration Score"

    plt.ylabel(f"{ylabel_text}", fontsize=18, labelpad=15)
    plt.title(title, fontsize=20, weight="bold" , pad=20)
    plt.xticks(rotation=90 if len(order) > 5 else 0, fontsize=14)
    plt.yticks(fontsize=14)

    sns.despine()
    plt.grid(False)

    # Save the plot as Pdf
    plt.tight_layout()
    plt.savefig(output_pdf, bbox_inches='tight', format='pdf', transparent=False)  # Fully vectorized PDF
    print(f"Figure saved as {output_pdf}")

# Generate plots
for category, pattern in CATEGORIES_FOR_VIOLIN.items():
    agent_count = category.split("_")[-2]  # Extract agent count dynamically

    if "different_story" in category:
        # Different story -> Cumulative Payoff
        csv_files = glob.glob(pattern)
        for csv_file in csv_files:
            vis_dir = get_visualization_directory("different_story")
            output_filename = f"cumulative_payoffs_{category}.pdf"
            output_path = vis_dir / output_filename

            df = preprocess_data(pd.read_csv(csv_file), "CumulativePayoff")
            title = f"Heterogenous Experiment"
            plot_violin(df, "CumulativePayoff", title, str(output_path))
    else:
        # Same story & bad apple -> Collaboration Score
        df = load_csv_files(pattern)
        df = preprocess_data(df, "CollaborationScore")
        if df is not None:
            exp_type = "same_story" if "same_story" in category else "bad_apple"
            vis_dir = get_visualization_directory(exp_type)
            output_filename = f"collaboration_violin_{category}.pdf"
            output_path = vis_dir / output_filename

            if "bad_apple" in category:
                title = f"Robustness"
            else:
                title = f"Homogenous Experiment"

            plot_violin(df, "CollaborationScore", title, str(output_path))

## 2. Scaling Analysis: Agent Size Effects
Analyze how cooperation changes with group size (4→16→32 agents).

Shows trajectory of each story's performance as groups scale up.

In [ ]:
def process_category_scaling(temp_label, CATEGORIES):
    """
    Analyze how cooperation changes with group size (scaling experiment).
    Creates trajectory plot showing each story's performance across agent sizes.
    Reveals whether cooperation patterns are robust to group scaling.
    """
    story_scores = {}

    # Load CSV data and extract mean collaboration scores
    for agent_count, pattern in CATEGORIES.items():

        df = load_csv_files(pattern)
        if df is None or df.empty:
            continue

        # Compute mean collaboration scores per story
        story_means = df.groupby("PromptType")["CollaborationScore"].mean()
        story_scores[agent_count] = story_means.to_dict()  # Store scores

    # Extract the order of stories as they appear at N=4 in ascending order
    starting_order = sorted(story_scores[4].items(), key=lambda x: x[1])
    starting_stories = [story for story, _ in starting_order]

    print(f"Processing {temp_label}: starting_stories = {starting_stories}")

    # Dynamically assign colors based on order in starting_stories
    COLOR_DICT_LOCAL = {}
    blue_idx, pink_idx = 0, 0  # Track index for blue and pink shades

    for story in starting_stories:
        if story in ["noinstruct", "nsCarrot", "nsPlumber", "maxreward"]:  # Baseline stories
            COLOR_DICT_LOCAL[story] = BLUE_SHADES[blue_idx]
            blue_idx += 1  # Move to next darker shade
        else:  # Meaningful stories
            COLOR_DICT_LOCAL[story] = PINK_SHADES[pink_idx]
            pink_idx += 1  # Move to next darker shade

    # Plot settings
    plt.figure(figsize=(12, 6))
    legend_handles = []

    # Plot each story's progression across agent sizes
    for story in COLOR_DICT_LOCAL.keys():  # Only plot stories in COLOR_DICT_LOCAL
        positions = []

        for agent_count in AGENT_SIZES:
            if agent_count in story_scores and story in story_scores[agent_count]:
                x_pos = story_scores[agent_count][story]
                y_pos = AGENT_SIZES.index(agent_count)
                positions.append((x_pos, y_pos))

                # Scatter plot for each point
                plt.scatter(x_pos, y_pos, s=70, facecolors="none", edgecolors=COLOR_DICT_LOCAL[story], linewidths=1.5, label=story if agent_count == 4 else "")

        if len(positions) > 1:
            x_vals, y_vals = zip(*positions)
            plt.plot(x_vals, y_vals, linestyle="dashed", color=COLOR_DICT_LOCAL[story], alpha=0.7)

    for story in starting_stories:
        legend_handles.append(
            mlines.Line2D(
                [], [], marker="o", linestyle="None", markersize=8, color=COLOR_DICT_LOCAL.get(story, "#888888"), label=story
            )
        )

    # Customizing plot
    plt.xlabel("Mean Collaboration Score", fontsize=18, labelpad=15)
    plt.ylabel("Agent Size", fontsize=18, labelpad=15)
    plt.yticks(range(len(AGENT_SIZES)), [f"N = {n}" for n in AGENT_SIZES], fontsize=14, weight="bold")
    plt.title(f"Scaling Experiment", fontsize=20, weight="bold", pad=20)
    plt.grid(axis="y", linestyle="dotted")

    plt.legend(
        handles=legend_handles, title="Story", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=12
    )

    sns.despine()
    plt.tight_layout()

    vis_dir = get_visualization_directory("same_story")
    filename = f"scaling_experiment_collab_score.pdf"
    output_path = vis_dir / filename
    plt.savefig(str(output_path), bbox_inches="tight", format="pdf")

    print(f"Scaling experiment figure saved as {output_path}")
    plt.show()

# Run the process for each category
for temp_label, category_dict in CATEGORY_GROUPS.items():
    process_category_scaling(temp_label, category_dict)


## 3. Summary Statistics Table
Generate a LaTeX-formatted table of mean ± std for final Collaboration Scores and Cumulative Payoffs across all story prompts.

In [ ]:
import re
import glob
import statistics
from pathlib import Path

def extract_agent_size(pattern):
    import re
    match = re.search(r'ag(\d+)', pattern)
    if match:
        return match.group(1)
    return "Unknown"

# File patterns
AGENT_SIZES = [4, 16, 32]
BASELINE_STORIES = ["noinstruct", "nsCarrot", "maxreward", "nsPlumber"]
MEANINGFUL_STORIES = ["OldManSons", "Odyssey", "Soup", "Peacemaker","Musketeers", "Teamwork", "Spoons", "Turnip"]

VISUALISATION_EXPERIMENTS = {
    "same_story": dict(
        sizes=AGENT_SIZES,
        pattern="experiments/single_pool/same_story/game_results_single_pool_same_story_*_ag{N}_ro5_end10_mult1.5.csv"
    ),
    "different_story": dict(
        sizes=[4],
        pattern="experiments/single_pool/different_story/game_results_single_pool_different_story_ag{N}_ro5_end10_mult1.5.csv"
    ),
    "bad_apple": dict(
        sizes=[4],
        pattern="experiments/single_pool/bad_apple/game_results_single_pool_bad_apple_*_ag{N}_ro5_end10_mult1.5.csv"
    ),
}

TABLE_KEYS = {
    "same_story": "homogeneous",
    "different_story": "heterogeneous",
    "bad_apple": "robustness",
}

CATEGORIES_FOR_TABLE = {
    TABLE_KEYS[name]: [
        VISUALISATION_EXPERIMENTS[name]["pattern"].format(N=N)
        for N in VISUALISATION_EXPERIMENTS[name]["sizes"]
    ]
    for name in VISUALISATION_EXPERIMENTS
}

# Initialize scores dictionary
COLLAB_SCORES = {story: {"Homogeneous_4": None, "Homogeneous_16": None, "Homogeneous_32": None,
                        "Robustness_4": None, "Heterogeneous_4": None}
                for story in BASELINE_STORIES + MEANINGFUL_STORIES}

# Data processing functions
def load_csv_files(pattern):
    """Loads all CSV files matching a pattern and merges them into a DataFrame."""
    files = glob.glob(pattern)
    if not files:
        print(f"No files found for pattern: {pattern}")
        return pd.DataFrame()
    return pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

def preprocess_data(df, metric):
    """Prepares data by filtering and converting columns to numeric types."""
    if df is None or df.empty:
        return None

    if metric == "CollaborationScore":
        df = df[df["Round"] == "final"].copy()
    else:  # "CumulativePayoff"
        df = df[df["Round"] != "final"].copy()
        df["Round"] = pd.to_numeric(df["Round"], errors="coerce")
        df.dropna(subset=["Round"], inplace=True)
        df = df.loc[df.groupby(["Game", "AgentName"])["Round"].idxmax()].copy()

    df[metric] = pd.to_numeric(df[metric], errors="coerce")
    df.dropna(subset=[metric], inplace=True)
    return df

# Process experiment data
print("Processing data for summary statistics table...")
data_found = False

for category, patterns in CATEGORIES_FOR_TABLE.items():
    for pattern in patterns:
        print(f"Checking pattern: {pattern}")

        df = load_csv_files(pattern)
        if df is None or df.empty:
            print(f"No data found for pattern: {pattern}")
            continue

        print(f"Loaded {len(df)} rows")
        data_found = True

        agent_size = extract_agent_size(pattern)
        print(f"Agent size extracted: '{agent_size}'")

        if category in ("homogeneous", "robustness"):
            metric = "CollaborationScore"
        else:
            metric = "CumulativePayoff"

        df_proc = preprocess_data(df, metric)
        if df_proc is None or df_proc.empty:
            print(f"No data after preprocessing")
            continue

        stats = df_proc.groupby("PromptType")[metric].agg(mean="mean", std="std").reset_index()
        column_key = f"{category.capitalize()}_{agent_size}"
        print(f"Mapping to column: {column_key}")

        for _, row in stats.iterrows():
            story = row["PromptType"]
            if story in COLLAB_SCORES:
                mean_val = row['mean']
                std_val = row['std']
                if std_val < 0.01:
                    formatted = f"{mean_val:.4f} ± {std_val:.4f}"
                else:
                    formatted = f"{mean_val:.2f} ± {std_val:.2f}"
                COLLAB_SCORES[story][column_key] = formatted
                print(f"Set {story} -> {column_key} = {formatted}")

# Generate LaTeX table if data found
if data_found:
    print("Generating LaTeX summary table...")

    latex_output = """\\begin{table*}[t]
        \\centering
        \\caption{Mean ± standard deviation of final Collaboration Scores (for homogeneous and robustness agents) and final Cumulative Payoffs (for heterogeneous agents) across all story prompts. Values are shown with higher decimal precision where variation is small, to reflect statistically meaningful differences observed in pairwise confidence intervals.}
        \\setlength{\\tabcolsep}{8pt}
        \\renewcommand{\\arraystretch}{1.3}
        \\fontsize{11pt}{13pt}\\selectfont
        \\resizebox{\\textwidth}{!}{
        \\begin{tabular}{llccccc}
            \\toprule
            \\multirow{2}{*}{\\textbf{Story Type}} & \\multirow{2}{*}{\\textbf{Story Prompt}} & \\multicolumn{3}{c}{\\textbf{Homogeneous Agents}} & \\textbf{Robustness} & \\textbf{Heterogeneous} \\\\
            \\cmidrule(lr){3-5} \\cmidrule(lr){6-6} \\cmidrule(lr){7-7}
            & & \\textbf{N=4} & \\textbf{N=16} & \\textbf{N=32} & \\textbf{N=4} & \\textbf{N=4} \\\\
            \\midrule
    """

    # Add Baseline Stories Section
    latex_output += "        \\multirow{4}{*}{\\centering \\textbf{Baseline Stories}}  \n"
    for story in BASELINE_STORIES:
        row_data = [COLLAB_SCORES[story].get(col, "N/A") or "N/A" for col in ["Homogeneous_4", "Homogeneous_16", "Homogeneous_32", "Robustness_4", "Heterogeneous_4"]]
        latex_output += f"        & {story}  & {' & '.join(row_data)} \\\\\n"

    latex_output += "        \\midrule\n"

    # Add Meaningful Stories Section
    latex_output += "        \\multirow{8}{*}{\\centering \\textbf{Meaningful Stories}}  \n"
    for story in MEANINGFUL_STORIES:
        row_data = [COLLAB_SCORES[story].get(col, "N/A") or "N/A" for col in ["Homogeneous_4", "Homogeneous_16", "Homogeneous_32", "Robustness_4", "Heterogeneous_4"]]
        latex_output += f"        & {story}  & {' & '.join(row_data)} \\\\\n"

    latex_output += """        \\bottomrule
        \\end{tabular}
        }
        \\label{tab:all_agents_scores}
    \\end{table*}
    """

    print(latex_output)

    # Save to visualization directory
    vis_dir = Path("experiments") / "vis" / "single_pool" / "same_story"
    vis_dir.mkdir(parents=True, exist_ok=True)
    table_path = vis_dir / "all_agents_table.tex"
    with open(table_path, "w") as f:
        f.write(latex_output)
    print(f"LaTeX table saved to: {table_path}")

else:
    print("No experimental data found. Run experiments first before generating summary table.")

# Multi-Pool Experiments – Visualisation
Visualize multi-pool results.

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
from pathlib import Path

# Story colors
COLOR_DICT = {
    "maxreward": "#87CEFA",
    "noinstruct": "#4682B4",
    "nsCarrot": "#4169E1",
    "nsPlumber": "#27408B",
    "Odyssey": "#FFB3E6",
    "Soup": "#FF99CC",
    "Peacemaker": "#FF66B3",
    "Musketeers": "#FF4D9E",
    "Teamwork": "#F02278",
    "Spoons": "#D81B60",
    "OldManSons": "#B22272",
    "Turnip": "#B83B7D"
}


# SAME STORY: Violin Plot (Collaboration Scores by Story)

def plot_same_story_violin_collaboration():
    """
    Violin plot of collaboration scores by story for multi-pool same story experiments.
    Shows how different stories perform when agents must allocate contributions
    across both global and local pools.
    """
    pattern = "experiments/multi_pool/same_story/game_results_multi_pool_2_same_story_*_ag4_ro5_end10_mult1.5.csv"
    csv_files = glob.glob(pattern)

    if not csv_files:
        print(f"No files found for pattern: {pattern}")
        return

    print(f"Found {len(csv_files)} files for multi-pool same story")

    # Combine all CSV files
    df_list = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        df_list.append(df)

    df_all = pd.concat(df_list, ignore_index=True)

    # Filter to final rows and get collaboration scores
    df_final = df_all[df_all["Round"] == "final"].copy()
    df_final["CollaborationScore"] = pd.to_numeric(df_final["CollaborationScore"], errors="coerce")
    df_final.dropna(subset=["CollaborationScore"], inplace=True)

    # Calculate means and order
    means = df_final.groupby("PromptType")["CollaborationScore"].mean()
    order = means.sort_values(ascending=True).index.tolist()

    # Create violin plot
    plt.figure(figsize=(12, 6))

    sns.violinplot(
        data=df_final,
        x="PromptType",
        y="CollaborationScore",
        order=order,
        palette=[COLOR_DICT.get(s, "#888888") for s in order],
        inner="box",
        scale="area",
        bw_adjust=5,
        cut=2
    )

    # Overlay mean trend line
    x_positions = np.arange(len(order))
    plt.plot(
        x_positions,
        means.loc[order].values,
        marker="o",
        linestyle="-",
        linewidth=2,
        alpha=0.5,
        color="black"
    )

    plt.title("Multi-Pool Homogeneous\n", fontsize=24, fontweight="bold")
    plt.ylabel("Collaboration Score", fontsize=22)
    plt.xlabel("Story Prompt", fontsize=22, labelpad=16)
    plt.xticks(rotation=90, ha="center", fontsize=22)
    plt.yticks(fontsize=18)
    plt.ylim(0, 1.0)

    sns.despine()
    plt.grid(False)
    plt.tight_layout()

    # Save plot
    output_dir = Path("experiments") / "vis" / "multi_pool" / "same_story"
    output_dir.mkdir(parents=True, exist_ok=True)
    out_path = output_dir / "multipool_same_story_violin_collaboration.pdf"
    plt.savefig(str(out_path), dpi=300, bbox_inches="tight")
    plt.show()
    plt.close()
    print(f"→ Saved same story violin plot to {out_path}")


# SAME STORY: Scatter Plot (Collaboration Score vs Global Pool Allocation)

def plot_same_story_scatter_collaboration():
    """Scatter plot of collaboration score vs global pool allocation for multi-pool same story."""
    pattern = "experiments/multi_pool/same_story/game_results_multi_pool_2_same_story_*_ag4_ro5_end10_mult1.5.csv"
    csv_files = glob.glob(pattern)

    if not csv_files:
        print(f"No files found for pattern: {pattern}")
        return

    df_list = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        df_list.append(df)

    df_all = pd.concat(df_list, ignore_index=True)

    # Calculate collaboration scores (from final rows)
    df_final = df_all[df_all["Round"] == "final"].copy()
    collab_scores = df_final.groupby("PromptType")["CollaborationScore"].mean()

    # Calculate global pool proportions (from non-final rows)
    df_rounds = df_all[df_all["Round"] != "final"].copy()
    df_rounds["GlobalContrib"] = pd.to_numeric(df_rounds["GlobalContrib"], errors="coerce")
    df_rounds["TotalContrib"] = pd.to_numeric(df_rounds["TotalContrib"], errors="coerce")

    global_props = {}
    for story in collab_scores.index:
        story_data = df_rounds[df_rounds["PromptType"] == story]
        if len(story_data) > 0:
            story_data["GlobalProp"] = story_data["GlobalContrib"] / story_data["TotalContrib"]
            story_data["GlobalProp"] = story_data["GlobalProp"].fillna(0.5)
            global_props[story] = story_data["GlobalProp"].mean()

    # Filter to stories with both metrics
    valid_stories = [s for s in collab_scores.index if s in global_props]
    xs = [global_props[s] for s in valid_stories]
    ys = [collab_scores[s] for s in valid_stories]

    # Create scatter plot
    plt.figure(figsize=(10, 7))
    colors = [COLOR_DICT.get(s, "#888888") for s in valid_stories]
    plt.scatter(xs, ys, s=120, c=colors, edgecolors="white", linewidths=1.5, alpha=1.0)
    plt.axvline(0.5, color="gray", linestyle="--", linewidth=1)

    # Create legend
    legend_elems = [
        Line2D([0], [0], marker="o", color="w", label=s,
               markerfacecolor=COLOR_DICT.get(s, "#888888"),
               markeredgecolor="white", markersize=10)
        for s in valid_stories
    ]
    plt.legend(handles=legend_elems, bbox_to_anchor=(1.05, 1), loc="upper left",
               title="Story", fontsize=18, title_fontsize=20, frameon=True)

    plt.title("Multi-Pool Homogeneous\nCollaboration Score vs Global-Pool Contribution Fraction",
              fontsize=20, fontweight="bold", pad=20)
    plt.xlabel("Proportion of Contributions to Global Pool", fontsize=22)
    plt.ylabel("Collaboration Score", fontsize=22)
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.grid(ls="--", alpha=0.5)
    plt.tight_layout()

    # Save plot
    output_dir = Path("experiments") / "vis" / "multi_pool" / "same_story"
    output_dir.mkdir(parents=True, exist_ok=True)
    out_path = output_dir / "multipool_same_story_scatter_collab_vs_global.pdf"
    plt.savefig(str(out_path), dpi=300, bbox_inches="tight", pad_inches=0.2)
    plt.show()
    plt.close()
    print(f"→ Saved same story scatter plot to {out_path}")



# DIFFERENT STORY: Violin Plot (Payoffs by Story)

def plot_different_story_violin_payoff():
    """Violin plot of payoffs by story for multi-pool different story experiments."""
    pattern = "experiments/multi_pool/different_story/game_results_multi_pool_2_different_story_ag4_ro5_end10_mult1.5.csv"
    csv_files = glob.glob(pattern)

    if not csv_files:
        print(f"No files found for pattern: {pattern}")
        return

    df_all = pd.read_csv(csv_files[0])

    # Filter to non-final rounds to get cumulative payoffs
    df_rounds = df_all[df_all["Round"] != "final"].copy()
    df_rounds["CumulativePayoff"] = pd.to_numeric(df_rounds["CumulativePayoff"], errors="coerce")
    df_rounds.dropna(subset=["CumulativePayoff"], inplace=True)

    # Get final payoffs (max round per agent per game)
    df_final_payoffs = df_rounds.loc[df_rounds.groupby(["Game", "AgentName"])["Round"].idxmax()].copy()

    # Remove 'All' if it exists
    df_final_payoffs = df_final_payoffs[df_final_payoffs["PromptType"] != "All"]

    # Calculate means and order
    means = df_final_payoffs.groupby("PromptType")["CumulativePayoff"].mean()
    order = means.sort_values(ascending=True).index.tolist()

    # Create violin plot
    plt.figure(figsize=(12, 6))

    sns.violinplot(
        data=df_final_payoffs,
        x="PromptType",
        y="CumulativePayoff",
        order=order,
        palette=[COLOR_DICT.get(s, "#888888") for s in order],
        inner="box",
        scale="area",
        bw_adjust=5,
        cut=2
    )

    # Overlay mean trend line
    x_positions = np.arange(len(order))
    plt.plot(
        x_positions,
        means.loc[order].values,
        marker="o",
        linestyle="-",
        linewidth=2,
        alpha=0.5,
        color="black"
    )

    plt.title("Multi-Pool Heterogeneous\n", fontsize=24, fontweight="bold")
    plt.ylabel("Average Cumulative Payoff", fontsize=22)
    plt.xlabel("Story Prompt", fontsize=22, labelpad=16)
    plt.xticks(rotation=90, ha="center", fontsize=22)
    plt.yticks(fontsize=18)
    plt.ylim(0, 200)  # Adjust based on your payoff range

    sns.despine()
    plt.grid(False)
    plt.tight_layout()

    # Save plot
    output_dir = Path("experiments") / "vis" / "multi_pool" / "different_story"
    output_dir.mkdir(parents=True, exist_ok=True)
    out_path = output_dir / "multipool_different_story_violin_payoff.pdf"
    plt.savefig(str(out_path), dpi=300, bbox_inches="tight")
    plt.show()
    plt.close()
    print(f"→ Saved different story violin plot to {out_path}")



# DIFFERENT STORY: Scatter Plot (Payoff vs Global Pool Allocation)

def plot_different_story_scatter_payoff():
    """Scatter plot of payoff vs global pool allocation for multi-pool different story."""
    pattern = "experiments/multi_pool/different_story/game_results_multi_pool_2_different_story_ag4_ro5_end10_mult1.5.csv"
    csv_files = glob.glob(pattern)

    if not csv_files:
        print(f"No files found for pattern: {pattern}")
        return

    df_all = pd.read_csv(csv_files[0])

    # Calculate payoffs (from non-final rounds, get final cumulative payoff per agent)
    df_rounds = df_all[df_all["Round"] != "final"].copy()
    df_final_payoffs = df_rounds.loc[df_rounds.groupby(["Game", "AgentName"])["Round"].idxmax()].copy()
    df_final_payoffs = df_final_payoffs[df_final_payoffs["PromptType"] != "All"]

    # Calculate mean payoffs by story
    payoff_means = df_final_payoffs.groupby("PromptType")["CumulativePayoff"].mean()

    # Calculate global pool proportions (from all non-final rounds)
    df_rounds["GlobalContrib"] = pd.to_numeric(df_rounds["GlobalContrib"], errors="coerce")
    df_rounds["TotalContrib"] = pd.to_numeric(df_rounds["TotalContrib"], errors="coerce")
    df_rounds = df_rounds[df_rounds["PromptType"] != "All"]

    global_props = {}
    for story in payoff_means.index:
        story_data = df_rounds[df_rounds["PromptType"] == story]
        if len(story_data) > 0:
            story_data["GlobalProp"] = story_data["GlobalContrib"] / story_data["TotalContrib"]
            story_data["GlobalProp"] = story_data["GlobalProp"].fillna(0.5)
            global_props[story] = story_data["GlobalProp"].mean()

    # Filter to stories with both metrics
    valid_stories = [s for s in payoff_means.index if s in global_props]
    xs = [global_props[s] for s in valid_stories]
    ys = [payoff_means[s] for s in valid_stories]

    # Create scatter plot
    plt.figure(figsize=(10, 7))
    colors = [COLOR_DICT.get(s, "#888888") for s in valid_stories]
    plt.scatter(xs, ys, s=120, c=colors, edgecolors="white", linewidths=1.5, alpha=1.0)
    plt.axvline(0.5, color="gray", linestyle="--", linewidth=1)

    # Create legend
    legend_elems = [
        Line2D([0], [0], marker="o", color="w", label=s,
               markerfacecolor=COLOR_DICT.get(s, "#888888"),
               markeredgecolor="white", markersize=10)
        for s in valid_stories
    ]
    plt.legend(handles=legend_elems, bbox_to_anchor=(1.05, 1), loc="upper left",
               title="Story", fontsize=18, title_fontsize=20, frameon=True)

    plt.title("Multi-Pool Heterogeneous\nPayoff vs Global-Pool Contribution Fraction",
              fontsize=20, fontweight="bold", pad=20)
    plt.xlabel("Proportion of Contributions to Global Pool", fontsize=22)
    plt.ylabel("Average Cumulative Payoff", fontsize=22)
    plt.xlim(0, 1)
    plt.ylim(0, max(ys) * 1.1 if ys else 100)  # Dynamic y-limit
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.grid(ls="--", alpha=0.5)
    plt.tight_layout()

    # Save plot
    output_dir = Path("experiments") / "vis" / "multi_pool" / "different_story"
    output_dir.mkdir(parents=True, exist_ok=True)
    out_path = output_dir / "multipool_different_story_scatter_payoff_vs_global.pdf"
    plt.savefig(str(out_path), dpi=300, bbox_inches="tight", pad_inches=0.2)
    plt.show()
    plt.close()
    print(f"→ Saved different story scatter plot to {out_path}")


# Run all 4 visualisastions

print("Generating ALL multi-pool visualizations...")

print("\\n=== 1. Same Story: Violin Plot (Collaboration Scores) ===")
plot_same_story_violin_collaboration()

print("\\n=== 2. Same Story: Scatter Plot (Collaboration vs Global Pool) ===")
plot_same_story_scatter_collaboration()

print("\\n=== 3. Different Story: Violin Plot (Payoffs) ===")
plot_different_story_violin_payoff()

print("\\n=== 4. Different Story: Scatter Plot (Payoff vs Global Pool) ===")
plot_different_story_scatter_payoff()

print("\\nAll multi-pool visualizations completed!")